In [6]:
!pip install findspark
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Colab PySpark Example") \
    .config("spark.jars.packages", "com.crealytics:spark-excel_2.12:0.14.0") \
    .getOrCreate()

Adidas sales data Analysis
The objective of this project is to analyze the Adidas sales database for the year 2020 and 2021 and identify key insights to help improve sales performance and optimize business strategies.

By analyzing the sales data, we aim to understand factors influencing sales, identify trends, and uncover opportunities for growth. The analysis will be conducted using databricks Notebook to provide an interactive and insightful dashboard.

Business Metrics requirements

Total Sales, Total Profit, Average Price per Unit, and Total Units Sold

Total sales by month

Total sales by state

Total sales by region

Total sales by product

Total sales by retailer

Units Sold by Product Category and Gender Type

Top Performing Cities by Profit

In [25]:
df = spark.read.csv("Adidas_sales_data.csv", header = True, inferSchema=True)
df.show()

+-------------+-----------+------------+---------+-----------+--------------+--------------------+--------------+----------+-----------+----------------+------------+
|     Retailer|Retailer ID|Invoice Date|   Region|      State|          City|             Product|Price per Unit|Units Sold|Total Sales|Operating Profit|Sales Method|
+-------------+-----------+------------+---------+-----------+--------------+--------------------+--------------+----------+-----------+----------------+------------+
|      Walmart|    1128299|   6/17/2021|Southeast|    Florida|       Orlando|     Women's Apparel|      $103.00 |       218|      2,245|         $1,257 |      Online|
|    West Gear|    1128299|   7/16/2021|    South|  Louisiana|   New Orleans|     Women's Apparel|      $103.00 |       163|      1,679|           $806 |      Online|
|Sports Direct|    1197831|   8/25/2021|    South|    Alabama|    Birmingham|Men's Street Foot...|       $10.00 |       700|      7,000|         $3,150 |      Outlet

convert it into temperory view

In [26]:
df.createOrReplaceTempView("sales_analysis")

In [28]:
df_spark = spark.sql("""SELECT * from sales_analysis """)
df_spark.show()

+-------------+-----------+------------+---------+-----------+--------------+--------------------+--------------+----------+-----------+----------------+------------+
|     Retailer|Retailer ID|Invoice Date|   Region|      State|          City|             Product|Price per Unit|Units Sold|Total Sales|Operating Profit|Sales Method|
+-------------+-----------+------------+---------+-----------+--------------+--------------------+--------------+----------+-----------+----------------+------------+
|      Walmart|    1128299|   6/17/2021|Southeast|    Florida|       Orlando|     Women's Apparel|      $103.00 |       218|      2,245|         $1,257 |      Online|
|    West Gear|    1128299|   7/16/2021|    South|  Louisiana|   New Orleans|     Women's Apparel|      $103.00 |       163|      1,679|           $806 |      Online|
|Sports Direct|    1197831|   8/25/2021|    South|    Alabama|    Birmingham|Men's Street Foot...|       $10.00 |       700|      7,000|         $3,150 |      Outlet

Total sales, Total Profit, average sales and Total unit sold

In [33]:
df_spark = spark.sql("""
  select sum(`Total Sales`) as Total_sales, sum(`Operating Profit`) as Total_Operating_Profit, avg(`Total Sales`) as Average_sales, sum(`Units Sold`) as Total_Unit_Sold from sales_analysis
""")
df_spark.show()

+-----------+----------------------+-----------------+---------------+
|Total_sales|Total_Operating_Profit|    Average_sales|Total_Unit_Sold|
+-----------+----------------------+-----------------+---------------+
|  2284311.0|                  NULL|463.2551206651795|      2418241.0|
+-----------+----------------------+-----------------+---------------+



 Total Sales by Month

In [37]:
total_sales = spark.sql("""
  select sum(`Total Sales`) as Total_sales, date_format(`Invoice Date`, 'MMMM') as Month
  from sales_analysis
  group by date_format(`Invoice Date`, 'MMMM')
  order by min(`Invoice Date`)
""")
total_sales.show()

+-----------+-----+
|Total_sales|Month|
+-----------+-----+
|  2284311.0| NULL|
+-----------+-----+



Based on State

In [41]:
total_bystate = spark.sql("""
  select sum(`Total Sales`) as Total_sales, State
  from sales_analysis
  group by state
  order by Total_sales desc
  limit 5
""")
total_bystate.show()

+-----------+----------+
|Total_sales|     State|
+-----------+----------+
|   145390.0|     Texas|
|   112765.0|California|
|    81382.0|   Georgia|
|    81365.0|New Mexico|
|    75104.0|   Arizona|
+-----------+----------+



Total sales by region

In [44]:
total_sales_byRegion = spark.sql("""
  select sum(`Total Sales`) as Total_sales, Region
  from sales_analysis
  group by Region
  order by Total_sales
""")
total_sales_byRegion.show()

+-----------+---------+
|Total_sales|   Region|
+-----------+---------+
|   291972.0|Southeast|
|   319413.0|  Midwest|
|   479034.0|    South|
|   511496.0|Northeast|
|   682396.0|     West|
+-----------+---------+



Total sales by Product


In [46]:
sales_byProduct = spark.sql("""
  select sum(`Total sales`) as Total_sales, Product
  from sales_analysis
  group by Product
""")
sales_byProduct.show()

+-----------+--------------------+
|Total_sales|             Product|
+-----------+--------------------+
|   377678.0|Men's Athletic Fo...|
|   335671.0|Women's Athletic ...|
|   359432.0|       Men's Apparel|
|   395111.0|     Women's Apparel|
|       NULL|        Men's aparel|
|   349721.0|Women's Street Fo...|
|   466698.0|Men's Street Foot...|
+-----------+--------------------+



Total sales by Retailer

In [49]:
sales_byretailer = spark.sql("""
  select sum(`Total Sales`) as Total_sales, Retailer
  from sales_analysis
  group by Retailer
  order by Total_sales

""")
sales_byretailer.show()

+-----------+-------------+
|Total_sales|     Retailer|
+-----------+-------------+
|   156984.0|      Walmart|
|   228509.0|       Amazon|
|   291675.0|       Kohl's|
|   525029.0|    West Gear|
|   531240.0|  Foot Locker|
|   550874.0|Sports Direct|
+-----------+-------------+



unit sold by category


In [50]:
sales_method = spark.sql("""
  select sum(`Total Sales`) as Total_sales , `Sales Method` as sales_method
  from sales_analysis
  group by sales_method
""")
sales_method.show()

+-----------+------------+
|Total_sales|sales_method|
+-----------+------------+
|     3250.0|    In-store|
|  1510964.0|      Online|
|   770097.0|      Outlet|
+-----------+------------+



Top performance

In [51]:
top_performance = spark.sql("""
 select sum(`Total Sales`) as Total_sales , City
 from sales_analysis
 group by City
 order by Total_sales desc

""")
top_performance.show()


+-----------+-----------+
|Total_sales|       City|
+-----------+-----------+
|    81382.0|    Atlanta|
|    81365.0|Albuquerque|
|    80897.0|    Houston|
|    76250.0|   Portland|
|    75104.0|    Phoenix|
|    75023.0|  Las Vegas|
|    73954.0|Los Angeles|
|    71736.0| Manchester|
|    69164.0|      Boise|
|    69136.0|  Knoxville|
|    65285.0| Burlington|
|    64493.0|     Dallas|
|    59395.0|    Jackson|
|    59167.0| Birmingham|
|    56120.0|    Orlando|
|    53331.0|   Hartford|
|    52868.0|   Richmond|
|    52321.0|     Boston|
|    51768.0|Little Rock|
|    51364.0| Charleston|
+-----------+-----------+
only showing top 20 rows

